# Preproces for **letters pairs**

##About
There is a task of decoding "crypted" text, which is based on the number of occurrences of pairs of letters in "natural language text".

This notebook is designed to pre-process the text to meet the requirements of the coding task, namely:
- the text contains only small letters and a space;
- all words write successively on one line - there are no paragraphs (\n) and other hidden characters.

Additional text processing is possible, which depends directly on the properties of a specific type of text.

---
Є задача декодування "зашифрованого" тексту, яка базується на кількості частоти входжень пар літер в "natural language text".

Цей ноутбук призначений для попередньої обробки тексту, щоб відповідати умовам задачі кодування, а саме:
- текст містить тільки маленьки літери та пробіл;
- всі слова їдуть підряд в один рядок -  відсутні абзаци (\n) та інші сховані символи.

Можлива додаткова обробка тексту, яка залежить беспосередньо від особливостей конкретного типу тексту.



The following texts were used for train:
- (1 HS) "Hamlet" by Shakespeare ([here](https://shakespearestudyguide.com/Hamlet%20Text.html)) [Without expressions like "BERNARDO:"]
- (2 TB) Torah, Sefer Bereshit in English ([here](https://www.tanach.us/Pages/About.html))
- (3 AR) Asimov "I, Robot"  ([here](https://royallib.com/book/Asimov_Isaac/I_Robot.html))
- (4 HP) "Harry Potter and the Philosopher's Stone" ([here](https://github.com/amephraim/nlp/blob/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt))
- (5 GG) Genetics : From Genes to Genomes. 6th edition ([here](http://skgjx.whu.edu.cn/Public/upfile/article/202103031656469260.pdf)) [Intro & parts 1,2,4. My edit from PDF]
- (6 SA) Scientific articles: from IEEE, [medium.com](https://medium.com/) etc.
- (7 NA) News articles: [New York Times](https://www.nytimes.com/international/), [Fox News](https://www.foxnews.com/), [BBC](https://www.bbc.com/news) etc.
- (8 CC) Coments in chats








В "Hamlet" окремо видалили слова на кшталт "BERNARDO:"
Тепер всі текстові файли овробляються однаково, за допомогою спеціальної функції.



---
---
## Import & mount

In [ ]:
import os
import re
import copy
from time import time
import numpy as np
import pandas as pd

# from matplotlib import pyplot as plt
# plt.rcParams['figure.figsize'] = [15, 6]

In [ ]:
# Mount GitHub
!git clone https://github.com/EdwardGerman/Columnar-Transposition-Cipher.git  # clone repository
%ls  # checking whether all files are present
drch = '/content/Columnar-Transposition-Cipher' # Path to data

folder_r = 'Data'
folder_w = 'Data_pp'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
---
## Funcfions

#### Space_n_letters()
Function for count of all letters and spaces in preproc text

In [ ]:
def Space_n_Letters(pp_text, p = False):
    chars_after_PP = len(pp_text)
    spaces = pp_text.count(' ')
    letters = chars_after_PP - spaces

    if p == False:
        return [chars_after_PP, letters, spaces]
    else:
        print('Symbols number:',chars_after_PP)
        print('Spaces number:',spaces)
        print('Only letters: ', letters,'\n')
    # print('Only letters (2): ', len(list(filter(str.isalpha, pp_text))),'\n')

#### CharacterCounts()
Counts the number of each character in the text and print sorted [by decreasing] list (dictionary).

!Noties! optiotal:
* Upper letter = lower letter
* Multyspace -> space

In [ ]:
# Returns sorted dictionary {letter : count} from largest to smallest number
def CharactersCount(orig_text, multispace = False, uppercase = False):
    text = copy.copy(orig_text)
    if multispace == False: text = re.sub(r' +', ' ', text)
    if uppercase  == False: text = text.lower()

    counts = {}   # Creating empty dictionary
    for char in text:    # Loop for count characters
        if char in counts: counts[char] += 1
        else:              counts[char]  = 1
    return dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

---
---
---
## Processing
Order is following:
- to lower letters;
- remove [apostrophe](https://en.wikipedia.org/wiki/Apostrophe) in the middle of the words;
- keep only letters and spaces (all rest convert to space);
- multispace to space;





In [ ]:
def TextPreprocessing(orig_txt):
    pp_text = copy.copy(orig_txt)

    pp_text = pp_text.lower()                                   # to lower letters
    pp_text = re.sub(r"(?<=[a-z])[`'’](?=[a-z])", '', pp_text)  # remove `'’

    pp_text = re.sub(r'[^a-z ]', ' ', pp_text)  # only letters and spaces (all rest convert to space)
    pp_text = re.sub(r' +', ' ', pp_text)       # multyspace to space

    if pp_text.endswith(' '): pp_text = pp_text[:-1]

    return pp_text

---

Get list of all files in directory

In [ ]:
# Get list of all files in directory
file_list = os.listdir(os.path.join(drch, 'Data'))
# file_list

Creating DataFrames for statistics

In [ ]:
chars_count_df = pd.DataFrame()                     # Df for count of all chars in texts
chars_count_pp_df = pd.DataFrame()                  # Df for count of chars in texts afrer preproc
                                                    # (only letters and spaces)
chars_number_df = pd.DataFrame(index=file_list,     # DF what contain chars number before & after PP
                               columns=['Chars before PP', 'Chars after PP', 'Letters', 'Spaces'])

Preprocessing in the loop:
- read text from file;
- count of chars in original text;
- exactly `Preprocessing`;
- count of chars in text preprocessing;
- write preprocessing text to file;



In [ ]:
all_pp_text = ''

# Loop for each file:
for text_file in file_list:
    file_name = text_file.replace('.txt', '')   # Only file name, without extension

    # Read:
    path_r = os.path.join(drch, folder_r, text_file)
    with open(path_r, 'r', errors='ignore') as file:
        orig_text = file.read()

    # Count of characters in original text (before pp):
    ser = pd.DataFrame([CharactersCount(orig_text)]).T
    ser.columns = [file_name]
    chars_count_df = pd.concat([chars_count_df, ser], axis=1)

    # Preprocessing process:
    pp_text = TextPreprocessing(orig_text)

    # Count of characters in preproc text:
    ser = pd.DataFrame([CharactersCount(pp_text)]).T
    ser.columns = [file_name]
    chars_count_pp_df = pd.concat([chars_count_pp_df, ser], axis=1)

    # Write:
    path_w = os.path.join(drch, folder_w, file_name + '_pp.txt')
    with open(path_w, 'w') as file:
        file.write(pp_text)

    # Count:
    chars_before_PP = len(orig_text)
    chars_number_df.loc[text_file] = [chars_before_PP] + Space_n_Letters(pp_text)

    # Add preproc text to "All text" variable
    all_pp_text = all_pp_text + ' ' + pp_text

Finishing of formation statistic DataFrames

In [ ]:
# --------------------------------------------------------------------------------------------------
# Finishing for "chars_count_df"
chars_count_df = chars_count_df.fillna(0)           # Change 'Nan' to '0'
chars_count_df = chars_count_df.astype(int)         # All values is integer

chars_count_df['Sum'] = chars_count_df.sum(axis=1)  # New column with sums for each chars in all files
chars_count_df = chars_count_df.sort_values(by='Sum', ascending=False)      # sort by count

# --------------------------------------------------------------------------------------------------
# Finishing for "chars_count_pp_df"
chars_count_pp_df = chars_count_pp_df.fillna(0)     # Change 'Nan' to '0'
chars_count_pp_df = chars_count_pp_df.astype(int)   # All values is integer

chars_count_pp_df['Sum'] = chars_count_pp_df.sum(axis=1)# New column with sums for each chars in all files
chars_count_pp_df = chars_count_pp_df.sort_values(by='Sum', ascending=False)# sort by count

# --------------------------------------------------------------------------------------------------
# Finishing for "chars_number_df"
chars_number_df.loc['All texts'] = chars_number_df.sum()

---

Write to text file all preprocessing texts

In [ ]:
all_pp_text = all_pp_text[1:]
path_w = os.path.join(drch, folder_w, 'All texts' + '_pp.txt')
with open(path_w, 'w') as file:
    file.write(all_pp_text)

## Statistics

#### 1. Characters number
- All chars number before preproc
- All chars number after preproc
- Letters number after preproc
- Spaces number after preproc

In [ ]:
chars_number_df

,Chars before PP,Chars after PP,Letters,Spaces
Torah_Bereshit.txt,199164,181245,144894,36351
Asimov_I_Robot.txt,400697,377897,307919,69978
Chat_Comments.txt,288735,277918,227565,50353
Harry_Potter_I.txt,439742,412898,335007,77891
Genetics_124.txt,396178,379158,316667,62491
Sci_articles.txt,360592,325498,268596,56902
News_articles.txt,101735,98258,81777,16481
Hamlet_BB.txt,242587,211013,170156,40857
All texts,2429430,2263885,1852581,411304


#### 2. Number of each character in each file `before preproc`

In [ ]:
pd.set_option('display.max_rows', None) # Print all rows (features) in DF
display(chars_count_df)
pd.reset_option('display.max_rows')     # Default setting: print print only first & final 5 rows

,Torah_Bereshit,Asimov_I_Robot,Chat_Comments,Harry_Potter_I,Genetics_124,Sci_articles,News_articles,Hamlet_BB,Sum
,37328,66953,47892,70803,56035,53178,16153,41949,390291
e,18716,37488,27325,39915,41548,32942,10076,20205,228215
t,12030,29480,21181,29048,26468,25495,6955,15720,166377
a,13194,23923,18257,26590,23792,21330,6961,12924,146971
o,11078,24728,18233,26141,25416,19496,6038,14297,145427
i,8831,21820,16753,20815,23969,20546,6265,11417,130416
n,8783,21648,15676,21825,22603,19138,5936,11086,126695
s,8770,18756,15257,19714,22440,18743,5551,11057,120288
r,7964,17367,13218,21650,19365,17301,5361,10063,112289
h,12078,17535,10999,22531,14595,10253,3599,10512,102102


#### 3. Number of each character in each file `after preproc`

In [ ]:
chars_count_pp_df['Sum'] = chars_count_pp_df.sum(axis=1)
chars_count_pp_df = chars_count_pp_df.sort_values(by='Sum', ascending=False)

chars_count_pp_df

,Torah_Bereshit,Asimov_I_Robot,Chat_Comments,Harry_Potter_I,Genetics_124,Sci_articles,News_articles,Hamlet_BB,Sum
,36351,69978,50353,77891,62491,56902,16481,40857,822608
e,18716,37488,27325,39915,41548,32942,10076,20205,456430
t,12030,29480,21181,29048,26468,25495,6955,15720,332754
a,13194,23923,18257,26590,23792,21330,6961,12924,293942
o,11078,24728,18233,26141,25416,19496,6038,14297,290854
i,8831,21820,16753,20815,23969,20546,6265,11417,260832
n,8783,21648,15676,21825,22603,19138,5936,11086,253390
s,8770,18756,15257,19714,22440,18743,5551,11057,240576
r,7964,17367,13218,21650,19365,17301,5361,10063,224578
h,12078,17535,10999,22531,14595,10253,3599,10512,204204


In [ ]:
lst = sorted(chars_count_pp_df.index.tolist())
print(lst)
# print(chars_count_pp_df.index.tolist().sort())

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


---

---
---
##Hamlet

### Bli "BERNARDO:"

In [ ]:
# drch = '/content/drive/MyDrive/DS small/Letters pairs/'

# path_r = os.path.join(drch, 'Data_add', 'Hamlet.txt')
# with open(path_r, 'r') as file:
#     orig_Hamlet = file.read()

# Hamlet_BB = re.sub(r'\b[A-Z]+\b:', '', orig_Hamlet)   # Regular expression for "BERNARDO:" etc.

# path_w = os.path.join(drch, 'Data_add', 'Hamlet_BB.txt')
# with open(path_w, 'w') as file:
#     file.write(Hamlet_BB)


---
---
---
#!!!
Draft version (this code don`t use in this project)

In [ ]:
# text   = "When I see the title, I was scared that it would required a NSFW tag, lol! It's safe to assume that's why we didn't get to see the basement of the Inn Seeing all those rabbits tells me this is moon guard."
# text_1 = "When I see the tit3le, I was sca4red that it w5ould required a 1NSFW tag, lo6l! It's safe to as9sume that'0s why w2e didn't get to see the bas34ement of the Inn See5ing all6 7those r0a0b8b7its tel6ls me th7is is moon gua8rd9."
# text_2 = "When I see the title, I was scared that it would required a NSFW tag, lol! It's safe to assume that's why we didn't get to see the basement of the Inn Seeing all those rabbits tells me this is moon guard."


In [ ]:
# text = "Це рядок 123 з кількома 4 цифрами в словах: b2a, a3bc, 1def, ghi4, eff56r, fgf34."
# # Використовуємо регулярний вираз для видалення цифр, які оточені буквами
# modified_text = re.sub(r'(?<=[a-zA-Z])\d(?=[a-zA-Z])', '', text)
# print(modified_text)

In [ ]:
# text = "Це рядок: b`a, ab~c, d'e!f, ghi~, ef'fr, fgf'."

# modified_text = re.sub(r"(?<=[a-zA-Z])[`~!'’](?=[a-zA-Z])", '', text)
# print(modified_text)

In [ ]:
# file_name = 'Torah_Bereshit_Bereshit.txt'
# folder = 'Data_add/'
# path = os.path.join(drch, folder, file_name)

# with open(path, 'r') as file:
#     orig_text = file.read()

# pp_text = copy.copy(orig_text)
# print(orig_text.count("’"))
# print(orig_text.count("'"))
# print(orig_text.count("‘"),'\n')


# pp_text = pp_text.lower()                                   # to lower letters
# pp_text = re.sub(r"(?<=[a-z])[`'’](?=[a-z])", '', pp_text)  # remove `'’
# print(pp_text)

# print(pp_text.count("’"))
# print(pp_text.count("'"))
# print(pp_text.count("‘"),'\n')

# file_name = 'Torah_Bereshit_Bereshit_pp.txt'
# folder = 'Data_add/'
# path = os.path.join(drch, folder, file_name)

# with open(path, 'w') as file:
#     file.write(pp_text)